In [1]:

import sqlite3 as sq

def sql_start():
    global base, cur
    base = sq.connect('post.db')
    cur = base.cursor()
    if base:
        print('DBase connected!')
    base.execute('CREATE TABLE IF NOT EXISTS users_main(post_id PRIMARY KEY,user_id INT, date_of_post VARCHAR, likes INT, comments INT )')
    base.execute('CREATE TABLE IF NOT EXISTS priznavashki(post_id PRIMARY KEY,user_id INT, date_of_post VARCHAR, likes INT, comments INT )')
    base.execute('CREATE TABLE IF NOT EXISTS likes_comments(user_id PRIMARY KEY, likes INT, comments INT )')
    base.commit()

def add_post(id,owner_id,date,likes,comments):
    cur.execute('INSERT INTO priznavashki VALUES (?,?,?,?,?)', (id,owner_id,date,likes,comments))
    base.commit()

def add_post_search(id,owner_id,date,likes,comments):
    cur.execute('INSERT INTO users_main VALUES (?,?,?,?,?)', (id,owner_id,date,likes,comments))
    base.commit()

def add_user_likes(id,likes,comments):
    cur.execute('INSERT INTO likes_comments VALUES (?,?,?)', (id,likes,comments))
    base.commit()

def check(data):
    dat = cur.execute('SELECT likes,comments FROM likes_comments WHERE user_id== ?',(data,)).fetchone()
    return dat

def add_something_blyat_uzhe_i_zaebalsya(user_id):
    cur.execute('UPDATE likes_comments SET likes=imp.likes+1 FROM( SELECT likes  FROM likes_comments WHERE user_id== ?) AS imp WHERE user_id== ?',(user_id,user_id))
    base.commit()

sql_start()


DBase connected!


In [3]:
import vk_api
from datetime import datetime
def auth_handler():
    key = input("Enter authentication code: ")
    remember_device = True
    return key, remember_device

def main():
    login, password = 'YOUR_PHONE', 'YOUR_PASSWORD'
    vk_session = vk_api.VkApi(login, password,auth_handler=auth_handler)

    try:
        vk_session.auth(token_only=True)
    except vk_api.AuthError as error_msg:
        print(error_msg)
        return

    tools = vk_api.VkTools(vk_session)

    wall = tools.get_all_iter('wall.get', 50, {'owner_id':-59518047 })

    for i in range(100000):
        post = next(wall)
        try:
            if post['signer_id']!=post['owner_id']:
                add_post(post['id'],post['signer_id'],datetime.fromtimestamp(post['date']).strftime('%Y-%m-%d'),post['likes']['count'],post['comments']['count'])
        except:
            add_post(post['id'],post['owner_id'],datetime.fromtimestamp(post['date']).strftime('%Y-%m-%d'),post['likes']['count'],post['comments']['count'])


if __name__ == '__main__':
    main()

KeyboardInterrupt: 

In [3]:
from time import sleep
import requests
from datetime import datetime
def get_data(method, params_method, offset=0, count=200,lenm=200):
    params = {
        'access_token': 'vk1.a.rjAJ3MCcjB6HI_Lk9Ala-MRkgeCaMDWJDFN-7VpyT0fWGBiRcD1pv_sYziEofBnVf-O_Ji8CLTWS2QaY_vNo4R2yFXWGKgUP8ylCL9OFtdIL8teqI27T5QM8uYqfR5tsudgamNqqwEZ3IWiDouziuM0w3tc8iI5qqBPLU9WTVvew-9WpI5OWmb9VmevQiRg6dn3n06hMnaseC6Lkf3iIgQ',
        'v': 5.131,
        'count': count,
        'offset': offset,
    }
    for key, item in params_method.items():
        params[key] = item
    data=[]
    response = requests.get(f'https://api.vk.com/method/{method}', params=params)
    try:
        response_data = response.json()['response']['items']
        data.extend(response_data)
    except KeyError:
        pass
        params['offset']+=params['count']
    if method == 'wall.get':
        print(f'Log: Get info from owner_id: {params["owner_id"]}. He has {len(data)} posts on a wall')
    return data

In [4]:
method = 'newsfeed.search'
params_search={
    'q':'spbu'
}
data = get_data(method,params_search)

for i,post in enumerate(data):
    try:
        add_post_search(post['id'],post['owner_id'],datetime.fromtimestamp(post['date']).strftime('%Y-%m-%d'),post['likes']['count'],post['comments']['count'])
    except:
        pass


{'access_token': 'vk1.a.rjAJ3MCcjB6HI_Lk9Ala-MRkgeCaMDWJDFN-7VpyT0fWGBiRcD1pv_sYziEofBnVf-O_Ji8CLTWS2QaY_vNo4R2yFXWGKgUP8ylCL9OFtdIL8teqI27T5QM8uYqfR5tsudgamNqqwEZ3IWiDouziuM0w3tc8iI5qqBPLU9WTVvew-9WpI5OWmb9VmevQiRg6dn3n06hMnaseC6Lkf3iIgQ', 'v': 5.131, 'count': 200, 'offset': 0, 'q': 'spbu'}
<Response [200]>


In [4]:
import pandas as pd
df = pd.read_sql_query("SELECT * from priznavashki", base)

In [5]:
df.groupby(['date_of_post'])['post_id'].count()

date_of_post
2013-10-17     56
2013-10-18    126
2013-10-19     66
2013-10-20     72
2013-10-21     78
             ... 
2023-04-08      8
2023-04-09      5
2023-04-10     10
2023-04-11      7
2023-04-12      4
Name: post_id, Length: 3387, dtype: int64

In [6]:
dff=df.groupby(['date_of_post'])['likes','comments'].sum()

C:\Users\Roman Andreevich\AppData\Local\Temp\ipykernel_20016\1142496527.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  dff=df.groupby(['date_of_post'])['likes','comments'].sum()


In [7]:
df.groupby(['user_id']).count().sort_values(['post_id'],ascending=False)

,post_id,date_of_post,likes,comments
user_id,,,,
-59518047,35582,35582,35582,35582
16712343,185,185,185,185
70547928,67,67,67,67
2093347,49,49,49,49
172355905,38,38,38,38
...,...,...,...,...
180377903,1,1,1,1
180342820,1,1,1,1
180266829,1,1,1,1


In [8]:
dfff = df.iloc[:1]

In [9]:
dfff

,post_id,user_id,date_of_post,likes,comments
0,409014,-59518047,2023-04-12,4,5


In [10]:

k=0
for id in dfff.post_id:
    k+=1
    print(k)
    params_get_likes = {
            'type': 'post',
            'owner_id': -59518047 ,
            'item_id': id,
            'friends_only': 0
        }
    method = 'likes.getList'
    data = get_data(method,params_get_likes)
    for i in data:
            add_something_blyat_uzhe_i_zaebalsya(id)






1


In [47]:
data

[306484457,
 136343027,
 199075643,
 155243998,
 217626487,
 172238821,
 471131514,
 383687679,
 26264645,
 230222255,
 472121925,
 386092147,
 507524517,
 393329095,
 297340164,
 172396788,
 307521265,
 448794855,
 352004717,
 208196914,
 549574850,
 242894406,
 272702726,
 217184015,
 210423604,
 198608288,
 214094861,
 197808392,
 253825010,
 454120167,
 328387649,
 99671800,
 331129578,
 557392996,
 512875824,
 158523487]